In [4]:
#The code snippet first crops the undesired portion of the image outside the A4 sheet on which the leaf is placed, thereby forming an image that leaves behind the blank sheet with its boundaries and the subject placed on it
import cv2
import numpy as np
import os

# Set the path to the folder containing images
folder_path = r"E:\TCV_project\Teacult\new_data\TV_11"

# Set the path for saving processed images
output_folder = r"E:\TCV_project\Teacult\new_data\TV_11_1"

# Create the output folder if it does not exist
os.makedirs(output_folder, exist_ok=True)

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    
    # Read the current image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve thresholding
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use adaptive thresholding to create a binary mask
    binary_mask = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                        cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours of the A4 sheet (largest contour)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours to find the one that most likely corresponds to the A4 sheet
    a4_contour = max(contours, key=cv2.contourArea)

    # Approximate the contour to get a polygon (4 corners)
    epsilon = 0.02 * cv2.arcLength(a4_contour, True)
    approx = cv2.approxPolyDP(a4_contour, epsilon, True)

    if len(approx) == 4:  # Check if we have found a quadrilateral
        # Order the points
        pts = approx.reshape(4, 2)
        rect = np.zeros((4, 2), dtype="float32")

        # Top-left point has the smallest sum, bottom-right has the largest sum
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]  # Top-left
        rect[2] = pts[np.argmax(s)]  # Bottom-right

        # Bottom-left will have the smallest difference, top-right has the largest difference
        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]  # Top-right
        rect[3] = pts[np.argmax(diff)]  # Bottom-left

        # Compute the width and height of the new image
        widthA = np.sqrt(((rect[2][0] - rect[3][0]) ** 2) + ((rect[2][1] - rect[3][1]) ** 2))
        widthB = np.sqrt(((rect[1][0] - rect[0][0]) ** 2) + ((rect[1][1] - rect[0][1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))

        heightA = np.sqrt(((rect[1][0] - rect[2][0]) ** 2) + ((rect[1][1] - rect[2][1]) ** 2))
        heightB = np.sqrt(((rect[0][0] - rect[3][0]) ** 2) + ((rect[0][1] - rect[3][1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))

        # Set destination points for the perspective transformation
        dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")

        # Compute the perspective transformation matrix
        M = cv2.getPerspectiveTransform(rect, dst)

        # Perform the perspective warp
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

        # Save the processed image
        output_path = os.path.join(output_folder, image_file)
        cv2.imwrite(output_path, warped)

        print(f"Processed and cropped image saved as: {output_path}")
    else:
        print(f"Could not find a quadrilateral in image: {image_file}")


Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120145942.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120145952.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120150023.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120150109.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120150123.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120150135.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120150215.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120150225.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120150247.jpg
Processed and cropped image saved as: E:\TCV_project\Teacult\new_data\TV_11_1\IMG20241120150301.jpg


In [21]:
#This snippet removes shadow to best possible extent for the tea leaf. Explanation for each step is given as a comment
import cv2
import numpy as np
import os

# Set the path to the folder containing processed images
input_folder_path = r"E:\TCV_project\Teacult\new_data\TV_13_A4"
# Set the path for saving leaf images
output_folder_path = r"E:\TCV_project\Teacult\new_data\TV_13_processed"

# Create the output folder if it does not exist
os.makedirs(output_folder_path, exist_ok=True)

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(input_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Define a wider range of green colors in HSV
lower_green = np.array([20, 37, 37])  # Slightly lower for darker greens
upper_green = np.array([95, 255, 255])  # Expanded upper range for lighter greens

for image_file in image_files:
    image_path = os.path.join(input_folder_path, image_file)

    # Read the current image
    image = cv2.imread(image_path)

    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a mask for the green colors
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Apply morphological operations to clean up the mask
    kernel_size = 15  # Slightly increased kernel size to handle leaf complexity
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Close small holes
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)  # Remove noise
    mask = cv2.erode(mask, kernel, iterations=2)  # Erode less aggressively to avoid cutting the leaf
    mask = cv2.dilate(mask, kernel, iterations=3)  # Increase dilation to sharpen edges

    # Find contours from the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an output image initialized to white
    output_image = np.ones_like(image) * 255

    if contours:
        # Get the largest contour (the leaf)
        largest_contour = max(contours, key=cv2.contourArea)

        # Approximate the contour to reduce the number of points
        epsilon = 0.0001 * cv2.arcLength(largest_contour, True)  # Keep the epsilon small for better fit
        approx_contour = cv2.convexHull(largest_contour)  # Using convexHull instead of approxPolyDP

        # Draw the contour on the output image
        cv2.drawContours(output_image, [approx_contour], -1, (0, 0, 0), thickness=cv2.FILLED)

        # Create a mask to keep the leaf interior unaffected
        mask = np.zeros_like(image)
        cv2.fillPoly(mask, [approx_contour], (255, 255, 255))

        # Combine the output image with the original image using the mask
        output_image = cv2.bitwise_or(output_image, cv2.bitwise_and(image, mask))

        # Further morphological operations to remove small white patches inside the leaf
        output_mask = cv2.cvtColor(output_image, cv2.COLOR_BGR2GRAY)
        output_mask[output_mask < 255] = 0  # Convert to binary mask (0s and 255s)

        # Find connected components in the mask
        num_labels, labels_im = cv2.connectedComponents(output_mask)

        # Keep only the largest component (the leaf)
        min_area_threshold = 1000  # Adjust as needed
        largest_area = 0
        largest_label = 0
        
        for label in range(1, num_labels):  # Skip the background label (0)
            area = np.sum(labels_im == label)
            if area > largest_area and area >= min_area_threshold:
                largest_area = area
                largest_label = label

        # Create a mask for the largest connected component
        final_mask = np.zeros_like(output_mask)
        final_mask[labels_im == largest_label] = 255

        # Use the final mask to get the final output image
        output_image = cv2.bitwise_or(output_image, cv2.cvtColor(final_mask, cv2.COLOR_GRAY2BGR))

        # Save the final image with the white background
        output_path = os.path.join(output_folder_path, image_file)
        cv2.imwrite(output_path, output_image)

        print(f"Leaf-only image saved as: {output_path}")
    else:
        print(f"No green contours found in image: {image_file}")


Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105719.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105749.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105810.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105831.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105852.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105906.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105921.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105933.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121105947.jpg
Leaf-only image saved as: E:\TCV_project\Teacult\new_data\TV_13_processed\IMG20241121110002.jpg
Leaf-only image saved as: E:\TCV_project